# Interferometry introduction

In [6]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


In [7]:
img_original = imread('shirt-picture.jpg')
img = ones((2048, 2048,3), dtype=img_original.dtype)*255
img[(2048-1884)//2:(2048-1884)//2+1884, 24:2024, :] = img_original

img_bw = img.mean(axis=2)[512:2048-512, 512:2048-512]
imsave(arr=img_bw, fname='shirt-bw.png')

In [8]:

#img_bw=img_bw[0:512, 256:768]

In [25]:
from ipywidgets import interact
import ipywidgets
from dataclasses import dataclass
#from scipy import fftpack
import pyfftw
import pyfftw.interfaces.scipy_fftpack as fftwpack
import numba
import numexpr

pyfftw.interfaces.cache.enable()
pyfftw.interfaces.cache.set_keepalive_time(keepalive_time=1000.0)

img_bw = pyfftw.empty_aligned((1024, 1024), dtype='float32')
img_bw[:] = array(imread('shirt-bw.png')[:,:,:3].mean(axis=2), dtype=float32)

@numba.njit('f4[:,:](f4[:,:])', parallel=True)
def dB(x):
    """
    Converts an array of positive, 32 bit floats to dB (10 log10(x)).

    Parameters
    ----------
    
    x : 2D ndarray of float32
        The numbers to convert.

    Returns
    -------
    2D ndarray of float32
    
    Examples
    --------
    >>> a = np.array([[0.1, 5, 100.0], [3, 0.01, 500]], dtype=np.float32)
    >>> dB(a)
    array([[-1.        ,  0.69897   ,  2.        ],
           [ 0.47712123, -2.        ,  2.69897   ]], dtype=float32)
    """
    
    nrows, ncols = x.shape
    result: numba.float32[:,:] = np.empty(x.shape, dtype=np.float32)
    for row in numba.prange(nrows):
        for col in range(ncols):
            result[row, col] = np.log10(x[row, col])
    return result




def dB_slow(x):
    return 10*numpy.log10(x)

def linear(x):
    return x

@numba.njit('f4[:,:](i8, i8, f8, f8)')
def min_max_bl_mask(num_y, num_x, min_bl_fraction, max_bl_fraction):
    mask = empty((num_y, num_x), dtype=float32)
    mid_y, mid_x = num_y//2, num_x//2
    max_r = sqrt(mid_x**2 + mid_y**2)
    minf2 = (min_bl_fraction*max_r)**2
    maxf2 = (max_bl_fraction*max_r)**2
    for y in prange(num_y):
        for x in range(num_x):
            r2 = (x-mid_x)**2 + (y - mid_y)**2
            if r2 >= minf2 and r2 <= maxf2:
                mask[y, x] = 1.0
            else:
                mask[y, x] = 0.0    
    return mask


@dataclass
class SimulationOutput:
    image      : numpy.ndarray
    uv_plane   : numpy.ndarray
    weights    : numpy.ndarray
    weighted_uv_plane: numpy.ndarray
    dirty_image: numpy.ndarray        


def simulate(image, weights):
    '''
    image : 2D numpy.array of floats
    
    mask : 2D numpy.array of floats
    '''
    w = weights
    uv_plane = fftwpack.fftshift(fftwpack.ifft2(image, threads=4))
    #print(image.dtype, w.dtype, uv_plane.dtype)
    weighted_uv_plane = pyfftw.empty_aligned(w.shape, dtype='complex64')
    weighted_uv_plane[:] = w*uv_plane
    dirty_image       = fftwpack.fft2(fftwpack.fftshift(weighted_uv_plane), threads=4).real
    #print(dirty_image.dtype)
    return SimulationOutput(image, uv_plane,
                            weights, weighted_uv_plane,
                           dirty_image)

In [21]:
fig = figure(dpi=100)
ax = fig.subplots(1,1)
ax.imshow(min_max_bl_mask(1024, 1024, 0.1, 0.707))

FigureCanvasNbAgg()

%%timeit
mask = min_max_bl_mask(1024, 1024, 0.1, 0.707)

mask = min_max_bl_mask(1024, 1024, 0.1, 0.707)

%%timeit
 _ = simulate(img_bw, mask)

%%timeit
_,_ = percentile(img_bw[::20,::20], [1,99])

In [26]:
%%timeit
dB(img_bw)

2.88 ms ± 66.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%%timeit
dB_slow(img_bw)

8.73 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
8.73/2.84

3.0739436619718314

In [12]:
#%%prun -s cumulative

fig = figure(figsize=(8,8),dpi=180)
((ax_tl, ax_tr), (ax_bl, ax_br)) = fig.subplots(2,2)
fig.subplots_adjust(hspace=0, wspace=0)
cbar_o = None
cbar_m = None
first_time = True

ax_tl_img = None
ax_tr_img = None
ax_bl_img = None
ax_br_img = None

previous_simresult = None

def replot(scale, img, min_bl_fraction, max_bl_fraction):
    global cbar_o, cbar_m
    global ax_tl_img, ax_tr_img, ax_bl_img, ax_br_img
    global previous_simresult
    mask = min_max_bl_mask(img[0].shape[0],img[0].shape[1] , min_bl_fraction/100.0, max_bl_fraction/100.0)
    
    sim_result = simulate(img[0], mask)
    # Original
    vmin, vmax = percentile(sim_result.image[::20,::20], [1, 99])
    if ax_tl_img is None:
        ax_tl_img = ax_tl.imshow(sim_result.image, cmap=cm.Greys_r, vmin=vmin, vmax=vmax,
                                interpolation='nearest')
        ax_tl.set_xticks([])
        ax_tl.set_yticks([])
    else:
        ax_tl_img.set_data(sim_result.image)
        ax_tl_img.set_clim(vmin, vmax)

    # Original uv plane
    to_plot = scale(numpy.absolute(sim_result.uv_plane))
    vmin, vmax = percentile(to_plot[::20,::20], (0.5, 99.0))
    if ax_tr_img is None:
        ax_tr_img = ax_tr.imshow(to_plot, vmax=vmax, vmin=vmin,
                                interpolation='nearest')
        ax_tr.set_xticks([])
        ax_tr.set_yticks([])
    else:
        ax_tr_img.set_data(to_plot)
        ax_tr_img.set_clim(vmin, vmax)

    # Masked uv plane
    to_plot = scale(numpy.absolute(sim_result.weighted_uv_plane))
    if ax_br_img is None:
        ax_br_img = ax_br.imshow(to_plot, vmax=vmax, vmin=vmin,
                                interpolation='nearest')
        ax_br.set_xticks([])
        ax_br.set_yticks([])
    else:
        ax_br_img.set_data(to_plot)
        ax_br_img.set_clim(vmin, vmax)
        
    # Dirty image
    vmin, vmax = percentile(sim_result.dirty_image[::20,::20], [1, 99])
    if ax_bl_img is None:
        ax_bl_img = ax_bl.imshow(sim_result.dirty_image, cmap=cm.Greys_r,
                                 vmin=vmin, vmax=vmax,
                                interpolation='nearest')
        ax_bl.set_xticks([])
        ax_bl.set_yticks([])
    else:
        ax_bl_img.set_data(sim_result.dirty_image)
        ax_bl_img.set_clim(vmin, vmax)
    previous_simresult = sim_result
    fig.canvas.draw_idle() # Apparently a bit faster than fig.canvas.draw()

#for _ in range(100):
#    replot(linear, (img_bw,), 10.0, 70.0)

interact(replot,
         scale={'Linear': linear, 'dB': dB},
         img={'Shirt': (img_bw,)},
        min_bl_fraction=ipywidgets.FloatSlider(min=0, max=100, step=0.25, value=0, continuous_update=True ),
        max_bl_fraction=ipywidgets.FloatSlider(min=0, max=100, step=0.25, value=100, continuous_update=True));

FigureCanvasNbAgg()

interactive(children=(Dropdown(description='scale', options={'Linear': <function linear at 0x7f2c9f15bd08>, 'd…

Source images: 

  - single point source
  - two point sources
  - three differen circular gausians
  - a field of random elliptical gausians
  - shirt


Example Telescopes:

  - circular min / max bl
  - WSRT
  - VLA
  - GMRT
  - LOFAR Core
  - LOFAR NL
  - LOFAR Intl
  
Implement ha_range, ha_0 (only NCP projection? Set Dec?)

Parameters: Image, Telescope, DEC, HA_0, HA_RANGE

Panels: orig      orig-uv
      : Dirty     uv masked with uv-cov
      
      
Simulate noise separately?


In [12]:
matplotlib.get_backend()

'module://ipympl.backend_nbagg'

In [4]:
help(imread)


Help on function imread in module matplotlib.pyplot:

imread(*args, **kwargs)
    Read an image from a file into an array.
    
    *fname* may be a string path, a valid URL, or a Python
    file-like object.  If using a file object, it must be opened in binary
    mode.
    
    If *format* is provided, will try to read file of that type,
    otherwise the format is deduced from the filename.  If nothing can
    be deduced, PNG is tried.
    
    Return value is a :class:`numpy.array`.  For grayscale images, the
    return array is MxN.  For RGB images, the return value is MxNx3.
    For RGBA images the return value is MxNx4.
    
    matplotlib can only read PNGs natively, but if `PIL
    <http://www.pythonware.com/products/pil/>`_ is installed, it will
    use it to load the image and return an array (if possible) which
    can be used with :func:`~matplotlib.pyplot.imshow`. Note, URL strings
    may not be compatible with PIL. Check the PIL documentation for more
    information.

